# Vector Search Explained - Location example
Dave Sisk, 2024/08/29

Let's start by listing the synonyms that vector search is also known by:
- Vector search (vector embeddings are the driver for this type of search)
- Vector similarity search (vector similarity is determined by distance calculations)
- Semantic search (vector embeddings try to capture the meaning of the text passage)
- AI search (it uses a large language model for vector embeddings)
- Cognitive search (search on meaning not keywords)
- Neural search (vector embeddings come from a neural network ML model)
- KNN search (K-nearest neighbors)
- ANN search (Approximate nearest neighbors)
- HNSW (Hierarchical Navicable Small World, an approximation algorithm for ANN)

## The basics
- What is a vector?
    - A set of numbers that describe somethign.  You can think of length, width, and height of a box as a 3-dimension vector.  Vector embeddings calculated by large language models are generally several hundred up to a couple thousand dimensions, depending on the purpose.  In this example, we'll examine geographic location as a 2-dimension vector.
- What is vector search?
    - Finding the top N nearest (or farthest) neighbors...semantically, they are most similar if their vector embeddings are near each other in high-dimensional space. 
- What is a vector database/datastore/index
    - A datastore that can 1) store the vector embeddings, but more importantly 2) do very efficient vector distance calculations by leveraging suitable approximation techniques.  This allows good vector databases to execute vector queries/vector search over billions of records with vector embeddings that might have thousands of dimensions, and deliver query responses in seconds or sub-seconds.

In this example, we'll use <b><i>Euclidean distance</i></b>.  Recalling the Pythagorean Theorem from middle school geometry, if you have a triangle with a 90 degree right angle, then the length of the longest side C can be calculated from the length's of the two shorter sides A and B based on this relationship:  C^2 = A^2 + B^2.  This means that, for our purposes, we can calculate the distance between two cities if we know their latitudes and longitudes by subtracting one latitude from the other and squaring it, subtracting one longitude from the other and squaring that, adding those two results together, then taking the square root of the sum...so, the square root of the sum of the squares of the differences between each pair of coordinates.  That's the Euclidean distance calculation, and it's one of the three distance calculations frequently used in vector search (euclidean, cosine, and dot-product).  For geographic distance, there are a few extra calcs, like converting degrees to radians, converting from radians to miles or kilometers, compensating for the elliptical shape of the earth, etc. There are of course numerous code libraries that already implement all this, so we'll simply use one of those instead of re-inventing the wheel. The take-home point is that the straight line distance calcs between two points are based on drawing an imaginary 90 degree triangle between.

In [6]:
#!pip install geopy # This package has a fn to calculate distance given two latitudes & longitudes
import geopy.distance

In [17]:
# Let's create a dictionary of a few cities with their latitude & longitude as a tuple
city_dict = {
    "Savannah GA":(32.076176, -81.088371),
    "St Augustine FL":(29.908072, -81.324791),
    "Hilton Head SC": (32.178890, -80.743057)
}
city_dict

{'Savannah GA': (32.076176, -81.088371),
 'St Augustine FL': (29.908072, -81.324791),
 'Hilton Head SC': (32.17889, -80.743057)}

In [30]:
# Decide on a origin and destination, and grab those geographic coordinates
# Let's say we want to drive from Savannah to St Augustine...let's find that as-the-crow-flies disstance
origin = city_dict["Savannah GA"]
destination = city_dict['St Augustine FL']
#destination = city_dict['Hilton Head SC']

print("The straight-line distance is", geopy.distance.geodesic(origin, destination).miles, "miles")

The straight-line distance is 150.02038684166016 miles


In [34]:
# Let's create a dictionary of international airport latitudes and longitudes
# We'll pre-filter the list to only same and nearby states...like a metadata filter in vector search
airport_dict = {
    "Savannah Intl": (32.1275833,-81.2021389),
    "Jacksonville Intl": (30.4940456,-81.6878467),
    "Charleston Intl": (32.8986389,-80.0405278),
    "Atlanta Intl": (33.6366996,-84.4278640),
    "Myrtle Beach Intl": (33.6797411,-78.9283214)
}
airport_dict

{'Savannah Intl': (32.1275833, -81.2021389),
 'Jacksonville Intl': (30.4940456, -81.6878467),
 'Charleston Intl': (32.8986389, -80.0405278),
 'Atlanta Intl': (33.6366996, -84.427864),
 'Myrtle Beach Intl': (33.6797411, -78.9283214)}

In [59]:
# If I'm in Savannah, let's find the distance from me to each airport
my_location = city_dict['Savannah GA']

# Calculate the distance from me to each airport
for row in airport_dict:
    dist = geopy.distance.geodesic(my_location, airport_dict[row]).miles
    print(f"Dist from me to {row} is {dist} miles")

Dist from me to Savannah Intl is 7.5544104403393755 miles
Dist from me to Jacksonville Intl is 114.6244466696679 miles
Dist from me to Charleston Intl is 83.40724388247548 miles
Dist from me to Atlanta Intl is 221.99809253039814 miles
Dist from me to Myrtle Beach Intl is 167.2865784430551 miles


In [60]:
# Let's do it again, except this time put the results into a dictionary for further manipulation
my_location = city_dict['Savannah GA']

dist_from_me_dict = {}  # Create an empty dict
# Calculate the distance from me to each airport
for row in airport_dict:
    dist = geopy.distance.geodesic(my_location, airport_dict[row]).miles # Calculate the distance
    dist_from_me_dict[row]=dist  # Add the key and value to the dictionary

# Now, let's sort the dictionary records by distance value ascending 
dist_from_me_dict = sorted(dist_from_me_dict.items(), key=lambda x: x[1])

# Finally, throw away all records EXCEPT the top 2 with the shortest distances
dist_from_me_dict = dist_from_me_dict[:2]

# Display the results
dist_from_me_dict



[('Savannah Intl', 7.5544104403393755), ('Charleston Intl', 83.40724388247548)]

<b><i>Congratulations, you have just done a manual KNN search with K=2!</u></b>  

This is easy and fast with a handful of records and vectors that only have 2 dimensions.  However, vector embeddings generated by LLM's (large language models) from full text generally have somewhere between 384 - 1576 dimensions...that's a few hundred times more calculations to perform.  And instead of only a handful of records, in practice we could be dealing with millions, billions, or trillions of records, which obviously multiplies the calculation burden by numerous orders of magnitude.  

This is why we need approximation techniques that still get us good enough results.  The most popular of those approximation algorithms is HNSW (Hierarchical Navicable Small World).  All of the currently available vector databases or indexing technologies have at least the HNSW algorithm available, sometimes others or customizations of this one. Pre-filtering of values via metadata filters or keyword filters can also create a smaller list to excecute the vector search against...this also helps to reduce the numeric processing burden. 

How is this an AI topic?  In practice, we typically find full text being handed to an LLM, then we catch the vector embedding that the LLM gives us back from that text, then we store that text and vector embedding in a vector store.  <b><i>The LMM provides a vector embedding that numerically reflects what the LLM interpreted the text to mean</i></b>...this is not just keyword matching, it's trying to capture the semantic meaning of the text. We might do searches with an incoming string that's a question or a simple body of long text...we do the same thing, hand that text to the LLM, catch the vector embedding it gives us for that text, then go search the vector DB for it's best (or sometimes worst) matches. This could be a recommendation system (which have been around for years), anomaly detection process, or it could be part of the RAG processing (Retrieval-Augmented Generation) in a Generative AI application, or any other use case that involves searching/querying/matching massive volumes of full text based on it's semantic meaning.

I hope you enjoyed this simple tour...thank you for your participation!